In [1]:
import pandas as pd
import numpy as np
import gc
import lightgbm as lgb
import seaborn as sns
import feature_engineering as fe
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
train=pd.read_csv("train.csv")

In [4]:
test=pd.read_csv("test.csv")

In [5]:
test.head()

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f289,f290,f291,f292,f293,f294,f295,f296,f297,label
0,20180120,0,0,1,1,100809.0,1,3,2,0,...,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0,2
1,20180105,1,1,0,0,100808.0,1,5,2,2,...,40.0,40.0,39.0,40.0,39.0,39.0,39.0,39.0,39.0,2
2,20180109,0,0,0,0,100803.0,0,6,1,1,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2
3,20180118,1,1,1,1,100808.0,0,3,2,1,...,301.0,302.0,122.0,301.0,208.0,208.0,207.0,89.0,207.0,2
4,20180109,1,1,1,1,100809.0,0,2,2,1,...,302.0,302.0,118.0,302.0,251.0,251.0,251.0,101.0,251.0,2


In [6]:
def dataProcess(data):
    data=fe.transfer_date2_week(data)
    data["f22_f21"]=data["f22"]-data["f21"]
    data["f24_f23"]=data["f24"]-data["f23"]
    data["f163_f162"]=data["f163"]-data["f162"]
    data["f164_f163"]=data["f164"]-data["f163"]
    data["f164_f162"]=data["f164"]-data["f162"]
    data.fillna(-999,inplace=True)
    return data

In [7]:
train=dataProcess(train)

In [9]:
test=dataProcess(test)

In [10]:
test=train[train.date>20171005]
train=train[train.date<20171006]

In [12]:
y=train["label"].values
test_y=test["label"].values
train.drop(["label","date"],axis=1,inplace=True)
test.drop(["label","date"],axis=1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [13]:
def get_cate_features(data):
    cat_col=[]
    cols = [c for c in data.columns if 'f' in c]
    for col in cols:
        if train[col].dtype =='int64':
            cat_col.append(col)
    return cat_col

In [41]:
categorical_feature=get_cate_features(train)
features=train.columns.tolist()

In [43]:
categorical_feature.append("isholiday")

In [44]:
categorical_feature

['f1',
 'f2',
 'f3',
 'f4',
 'f6',
 'f7',
 'f8',
 'f9',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'isholiday']

In [16]:
kkk=pd.concat([train,test])

In [20]:
for item in kkk.columns:
    if kkk[item].nunique()<=10:
        categorical_feature.append(item)

In [23]:
train_x,valid_x,train_y,valid_y=train_test_split(train,y,test_size=0.2,random_state=2018)

In [45]:
clf=lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.05,
    n_estimators=1000,
    max_bin=255,
    objective=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=1,
    colsample_bytree=1.0,
    random_state=None, 
    n_jobs=-1,
    silent=True
)
clf.fit(train_x, train_y,
         eval_set=[(valid_x,valid_y)],
        eval_metric='auc',
        early_stopping_rounds=100, verbose=True, feature_name=features,
        categorical_feature=categorical_feature, callbacks=None
        )

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['f1', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f2', 'f3', 'f4', 'f6', 'f7', 'f8', 'f9', 'isholiday']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's auc: 0.909329
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.926688
[3]	valid_0's auc: 0.926931
[4]	valid_0's auc: 0.926556
[5]	valid_0's auc: 0.926531
[6]	valid_0's auc: 0.927301
[7]	valid_0's auc: 0.943033
[8]	valid_0's auc: 0.943393
[9]	valid_0's auc: 0.943408
[10]	valid_0's auc: 0.943236
[11]	valid_0's auc: 0.943641
[12]	valid_0's auc: 0.944438
[13]	valid_0's auc: 0.944667
[14]	valid_0's auc: 0.944818
[15]	valid_0's auc: 0.945443
[16]	valid_0's auc: 0.945638
[17]	valid_0's auc: 0.945797
[18]	valid_0's auc: 0.945785
[19]	valid_0's auc: 0.945656
[20]	valid_0's auc: 0.945657
[21]	valid_0's auc: 0.945617
[22]	valid_0's auc: 0.945621
[23]	valid_0's auc: 0.945637
[24]	valid_0's auc: 0.945931
[25]	valid_0's auc: 0.946115
[26]	valid_0's auc: 0.946446
[27]	valid_0's auc: 0.946623
[28]	valid_0's auc: 0.946643
[29]	valid_0's auc: 0.946787
[30]	valid_0's auc: 0.947585
[31]	valid_0's auc: 0.947642
[32]	valid_0's auc: 0.959015
[33]	valid_0's auc

[277]	valid_0's auc: 0.98982
[278]	valid_0's auc: 0.989815
[279]	valid_0's auc: 0.989826
[280]	valid_0's auc: 0.989816
[281]	valid_0's auc: 0.989811
[282]	valid_0's auc: 0.989806
[283]	valid_0's auc: 0.989814
[284]	valid_0's auc: 0.989827
[285]	valid_0's auc: 0.98984
[286]	valid_0's auc: 0.989845
[287]	valid_0's auc: 0.989844
[288]	valid_0's auc: 0.989847
[289]	valid_0's auc: 0.98985
[290]	valid_0's auc: 0.989868
[291]	valid_0's auc: 0.989896
[292]	valid_0's auc: 0.989891
[293]	valid_0's auc: 0.989896
[294]	valid_0's auc: 0.989899
[295]	valid_0's auc: 0.989907
[296]	valid_0's auc: 0.989902
[297]	valid_0's auc: 0.989913
[298]	valid_0's auc: 0.989918
[299]	valid_0's auc: 0.989923
[300]	valid_0's auc: 0.98993
[301]	valid_0's auc: 0.989932
[302]	valid_0's auc: 0.989933
[303]	valid_0's auc: 0.98994
[304]	valid_0's auc: 0.989946
[305]	valid_0's auc: 0.989971
[306]	valid_0's auc: 0.990003
[307]	valid_0's auc: 0.990026
[308]	valid_0's auc: 0.990026
[309]	valid_0's auc: 0.99004
[310]	valid_0's 

[551]	valid_0's auc: 0.990727
[552]	valid_0's auc: 0.990733
[553]	valid_0's auc: 0.990732
[554]	valid_0's auc: 0.990734
[555]	valid_0's auc: 0.990736
[556]	valid_0's auc: 0.990739
[557]	valid_0's auc: 0.990741
[558]	valid_0's auc: 0.990749
[559]	valid_0's auc: 0.990739
[560]	valid_0's auc: 0.990743
[561]	valid_0's auc: 0.990739
[562]	valid_0's auc: 0.990741
[563]	valid_0's auc: 0.990735
[564]	valid_0's auc: 0.99073
[565]	valid_0's auc: 0.990726
[566]	valid_0's auc: 0.990733
[567]	valid_0's auc: 0.990728
[568]	valid_0's auc: 0.990741
[569]	valid_0's auc: 0.990746
[570]	valid_0's auc: 0.990743
[571]	valid_0's auc: 0.990743
[572]	valid_0's auc: 0.99074
[573]	valid_0's auc: 0.990743
[574]	valid_0's auc: 0.990755
[575]	valid_0's auc: 0.99076
[576]	valid_0's auc: 0.990759
[577]	valid_0's auc: 0.990758
[578]	valid_0's auc: 0.99076
[579]	valid_0's auc: 0.990774
[580]	valid_0's auc: 0.990772
[581]	valid_0's auc: 0.990769
[582]	valid_0's auc: 0.990768
[583]	valid_0's auc: 0.990776
[584]	valid_0'

[826]	valid_0's auc: 0.99119
[827]	valid_0's auc: 0.991196
[828]	valid_0's auc: 0.991204
[829]	valid_0's auc: 0.991204
[830]	valid_0's auc: 0.991204
[831]	valid_0's auc: 0.991208
[832]	valid_0's auc: 0.991198
[833]	valid_0's auc: 0.991207
[834]	valid_0's auc: 0.991211
[835]	valid_0's auc: 0.991215
[836]	valid_0's auc: 0.991218
[837]	valid_0's auc: 0.991224
[838]	valid_0's auc: 0.991224
[839]	valid_0's auc: 0.991228
[840]	valid_0's auc: 0.991232
[841]	valid_0's auc: 0.991233
[842]	valid_0's auc: 0.991235
[843]	valid_0's auc: 0.991231
[844]	valid_0's auc: 0.991231
[845]	valid_0's auc: 0.991236
[846]	valid_0's auc: 0.991237
[847]	valid_0's auc: 0.991238
[848]	valid_0's auc: 0.991244
[849]	valid_0's auc: 0.991248
[850]	valid_0's auc: 0.991251
[851]	valid_0's auc: 0.991253
[852]	valid_0's auc: 0.991254
[853]	valid_0's auc: 0.991261
[854]	valid_0's auc: 0.991261
[855]	valid_0's auc: 0.991261
[856]	valid_0's auc: 0.991254
[857]	valid_0's auc: 0.991257
[858]	valid_0's auc: 0.991254
[859]	valid

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.05, max_bin=255, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [46]:
proba=clf.predict_proba(test)

In [47]:
from sklearn.metrics import roc_curve
def score(y_true, y_score):
    """ Evaluation metric
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    score = 0.4 * tpr[np.where(fpr >= 0.001)[0][0]] + \
            0.3 * tpr[np.where(fpr >= 0.005)[0][0]] + \
            0.3 * tpr[np.where(fpr >= 0.01)[0][0]]

    return score

In [48]:
scores=score(test_y,proba[:,1])

In [49]:
scores

0.44736258514177096

In [29]:
clf.feature_importances_

array([ 39,  22,  24,  69, 154,  75, 244,  22,  50,  30,  41,  33,  32,
       105, 109,  33,  52,  87,  60,  18,  27,  36,  29,  55,  62,  51,
        60,  84,  81,  87,  75, 109,  93,  37,  20,  21,  17,  17,  16,
         5,   6,   8,   9,  17,   3,   2,   5,  22,  18,  24,  18,   1,
         0,   1,   8,  30,  37,  61,   8,  11,   0,   0,   0,  16,  20,
        20,  10,   4,  15,  21,   9,   8,   1,  14,  17,  12,  97,  59,
        49,  73,  65, 115, 207,  16,  12,  26,  19, 106,  50,  45,  87,
        12,  26,  23,  35,  20,  29,  27,  27,  16,  25,  27,  35,  38,
        50,  42, 138,  87,  79,  64, 147,  36,  34,  45,  51,  85,  92,
        86,  70,  60, 124,  57,  40,  41,  40, 120,   3,   6,   0,   7,
         7,  49,  39,  40,  88,  54,  30,  25,  46,   1,   8,  11,  37,
        28,   8,   1,  18,  17,  14,  27,  74,  39,  31,  27,  35,  57,
        41,  22,  29,  45,  14,  52,  51,  85,  18,  24,  20,  21,  46,
        33,  37,  49,  80,   0,  39,  48,  48])

In [31]:
feature_import=pd.DataFrame()

In [32]:
feature_import["feature_name"]=features

In [33]:
feature_import["importance"]=clf.feature_importances_

In [35]:
feature_import.to_csv("feature_importance.csv",index=False)